In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[ 0.0749,  0.1913, -0.0843,  ...,  0.0644, -0.0095,  0.0832],
        [ 0.0302,  0.2408, -0.0676,  ...,  0.0820, -0.0421, -0.0459],
        [-0.0196,  0.0806, -0.2156,  ...,  0.0997, -0.0298, -0.0109],
        ...,
        [-0.0626,  0.1346, -0.0490,  ...,  0.1283, -0.1131, -0.1170],
        [ 0.0553,  0.1540, -0.1084,  ...,  0.0288, -0.0624,  0.0552],
        [ 0.0066,  0.1153, -0.1691,  ...,  0.0531,  0.0709, -0.0008]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 150
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-22.1983, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.3797, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8504, device='cuda:0', grad_fn=<NegBackward0>), tensor(-51.8933, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-13.7949, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.8091, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5118, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.2641, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-13.7949, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.8091, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5118, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.2641, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-24.4406, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-10.0354, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.4269, device='cuda:0', grad_fn=<NegBackward0>), tensor(-52.4294, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-24.4406, device='cuda:0', grad_fn=<NegBackward0>) [ten

tensor(-44.5368, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.5381, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.6431, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.9113, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-44.5368, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.5381, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.6431, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.9113, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-45.2962, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.8640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-18.0806, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.5984, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-45.2962, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.8640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-18.0806, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.5984, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-45.2532, device='cuda:0', grad_fn=<NegBackward0>

tensor(-53.0084, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.4822, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.0943, device='cuda:0', grad_fn=<NegBackward0>), tensor(-83.0941, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-52.4717, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.0753, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.9745, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.2585, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-52.4717, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.0753, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.9745, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.2585, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-53.0756, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.4760, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.1724, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.7015, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-53.0756, device='cuda:0', grad_fn=<NegBackward0>

tensor(-58.6812, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.0608, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.1427, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.0252, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.4228, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.3009, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.6354, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.8105, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.4228, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.3009, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.6354, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.8105, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.8610, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.5056, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.8659, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.0720, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.8610, device='cuda:0', grad_fn=<NegBackward0>

tensor(-63.1481, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.9960, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.6396, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.1439, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.1481, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.9960, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.6396, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.1439, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-64.4661, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.6047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3372, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.2242, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-64.4661, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.6047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3372, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.2242, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-65.5117, device='cuda:0', grad_fn=<NegBackward0>

tensor(-68.9585, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.6037, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.7969, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.3844, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.0743, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.6544, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.8616, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.4238, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.0743, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.6544, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.8616, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.4238, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.6314, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.4491, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.6305, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.8054, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.6314, device='cuda:0', grad_fn=<NegBackward0>

tensor(-68.5753, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.4959, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.5357, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.0357, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.5753, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.4959, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.5357, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.0357, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.2670, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.8325, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.8822, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.8497, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.2670, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.8325, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.8822, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.8497, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.0231, device='cuda:0', grad_fn=<NegBackward0>

tensor(-70.5068, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.5060, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.4444, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.2356, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-71.7463, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.9185, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.2623, device='cuda:0', grad_fn=<NegBackward0>), tensor(-95.1176, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-71.7463, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.9185, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.2623, device='cuda:0', grad_fn=<NegBackward0>), tensor(-95.1176, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-72.8752, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-35.5047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.7953, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.0420, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-72.8752, device='cuda:0', grad_fn=<NegBackward0>

tensor(-74.2780, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.0582, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.6370, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.7862, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-74.2780, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.0582, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.6370, device='cuda:0', grad_fn=<NegBackward0>), tensor(-96.7862, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-74.5836, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.2272, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.7697, device='cuda:0', grad_fn=<NegBackward0>), tensor(-97.1599, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-74.5836, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.2272, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.7697, device='cuda:0', grad_fn=<NegBackward0>), tensor(-97.1598, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-73.9469, device='cuda:0', grad_fn=<NegBackward0>

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05it/s]


----------------------------
Train_z, Test_z
PCA: 0.8443
SVD: 0.8429
NetD, Acc at first 0.8681
